In [1]:
import os

In [2]:
%pwd

'/home/kalema/Projects/MIIA-Pothole-Image-classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/kalema/Projects/MIIA-Pothole-Image-classification'

In [11]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    """Configuration class for preparing base models.
    
    Attributes:
        root_dir (Path): The root directory where model-related files are stored.
        base_model_path (Path): The path where the base model will be saved.
        params_image_size (list): A list representing the image size parameters.
        params_classes (int): The number of classes in the model.
        params_dense_units (int): The number of neurons  in the fully connected layer
        params_conv_1_filters (int): The number of filters in the first convolutional layer
        params_conv_2_filters (int): The number of filters in the second convolutional layer
        params_conv_3_filters (int): The number of filters in the third convolutional layer
        params_conv_4_filters (int): The number of filters in the fourth convolutional layer
    """
    root_dir: Path
    base_model_path: Path
    params_image_size: list
    params_classes: int
    params_dense_units: int
    params_conv_1_filters: int
    params_conv_2_filters: int
    params_conv_3_filters: int
    params_conv_4_filters: int

In [12]:
from potholeClassifier.constants import *
from potholeClassifier.utils.common import read_yaml, create_directories

In [13]:
from pathlib import Path

class ConfigurationManager:
    """Class for managing configuration files and preparing base models.
    
    This class handles the loading of configuration files and parameters,
    as well as the creation of directories necessary for preparing base models.
    
    Attributes:
        config_filepath (str, optional): The filepath of the configuration file. Defaults to CONFIG_FILE_PATH.
        params_filepath (str, optional): The filepath of the parameters file. Defaults to PARAMS_FILE_PATH.
    
    Methods:
        get_prepare_base_model_config(): Retrieves the configuration for preparing base models.
    """

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """Initializes the ConfigurationManager.

        Args:
            config_filepath (str, optional): The filepath of the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): The filepath of the parameters file. Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_base_model_config(self) -> PrepareBaseModelConfig:
        """Retrieves the configuration for preparing base models.

        Returns:
            PrepareBaseModelConfig: The configuration for preparing base models.
        """
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_dense_units=self.params.DENSE_UNITS,
            params_conv_1_filters=self.params.CONV_1_FILTERS,
            params_conv_2_filters=self.params.CONV_2_FILTERS,
            params_conv_3_filters=self.params.CONV_3_FILTERS,
            params_conv_4_filters=self.params.CONV_4_FILTERS,
            params_classes=self.params.CLASSES
        )

        return base_model_config


In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

class PrepareBaseModel:
    """Class for preparing base models.

    This class provides methods preparing a full model and saving it to a specified path.

    Attributes:
        config (PrepareBaseModelConfig): The configuration for preparing base models.
    """

    def __init__(self, config: PrepareBaseModelConfig):
        """Initializes the PrepareBaseModel.

        Args:
            config (PrepareBaseModelConfig): The configuration for preparing base models.
        """
        self.config = config

    @staticmethod
    def _prepare_full_model(conv_1_filters, conv_2_filters, conv_3_filters, conv_4_filters, dense_units, number_of_classes, image_size):
        """Prepares the full model by freezing specified layers and adding additional layers.

        Args:
            number_of_classes (int): The number of classes in the model.
            conv_1_filters: Number of filters on the first convolutional layer
            conv_2_filters: Number of filters on the second convolutional layer
            conv_3_filters: Number of filters on the third convolutional layer
            conv_4_filters: Number of filters on the fourth convolutional layer
            dense_units: Number of neurons in the fully connected dense layer
            image_size: Input shape of the model

        Returns:
            tf.keras.Model: The prepared full model.
        """
        full_model = Sequential([
            Conv2D(conv_1_filters, (3, 3), activation='relu', input_shape=image_size),
            MaxPooling2D((2, 2)),
            Conv2D(conv_2_filters, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(conv_3_filters, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(conv_4_filters, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(dense_units, activation='relu'),
            Dense(number_of_classes, activation='softmax')
        ])

        # Compile the model
        full_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Print the model summary
        full_model.summary()
        return full_model
    
    def _save_base_model(self):
        self.full_model = self._prepare_full_model(
            number_of_classes=self.config.params_classes,
            dense_units=self.config.params_dense_units,
            conv_1_filters=self.config.params_conv_1_filters,
            conv_2_filters=self.config.params_conv_2_filters,
            conv_3_filters=self.config.params_conv_3_filters,
            conv_4_filters=self.config.params_conv_4_filters,
            image_size=self.config.params_image_size
        )

        self._save_model(path=self.config.base_model_path, model=self.full_model)

    @staticmethod
    def _save_model(path: Path, model: tf.keras.Model):
        """Saves the model to the specified path.

        Args:
            path (Path): The path where the model will be saved.
            model (tf.keras.Model): The model to be saved.
        """
        model.save(path)


In [17]:
try: 
    config = ConfigurationManager()
    base_model_config = config.get_base_model_config()
    base_model = PrepareBaseModel(config=base_model_config)
    base_model._save_base_model()
except Exception as e:
    raise e

[2024-04-29 10:24:53,039: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-29 10:24:53,050: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-29 10:24:53,055: INFO: common: Created directory at: artifacts]
[2024-04-29 10:24:53,058: INFO: common: Created directory at: artifacts/base_model]
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 111, 111, 64)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 54, 54, 128)   